# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_total_quantity= data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
df_total_quantity.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
product_customer_matrix= pd.pivot_table(df_total_quantity, index="ProductName", columns='CustomerID', values="Quantity").fillna(0)
product_customer_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
cs_matrix= squareform(pdist(product_customer_matrix.T, 'cosine')) #euclidean
cs_matrix

array([[0.        , 0.88678126, 0.77464789, ..., 0.9223069 , 0.88864251,
        0.89702036],
       [0.88678126, 0.        , 0.86162154, ..., 0.8612137 , 0.90053882,
        0.89652438],
       [0.77464789, 0.86162154, 0.        , ..., 0.85756266, 0.80203113,
        0.89702036],
       ...,
       [0.9223069 , 0.8612137 , 0.85756266, ..., 0.        , 0.84074449,
        0.88165473],
       [0.88864251, 0.90053882, 0.80203113, ..., 0.84074449, 0.        ,
        0.83037577],
       [0.89702036, 0.89652438, 0.89702036, ..., 0.88165473, 0.83037577,
        0.        ]])

In [7]:
customer_similarity_matrix = pd.DataFrame(1/(1 + cs_matrix), 
                         index=product_customer_matrix.columns, columns=product_customer_matrix.columns)

customer_similarity_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
top5 = customer_similarity_matrix.nlargest(6, [33])[1:6]
top5

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
27672,0.575128,0.529414,0.533165,0.538077,0.530522,0.535919,0.537163,0.553129,0.539442,0.544386,...,0.523105,0.529219,0.530719,0.550934,0.542963,0.545308,0.525264,0.536841,0.554546,0.546613
45549,0.574835,0.543218,0.525190,0.520277,0.544890,0.520980,0.544704,0.537944,0.528369,0.557190,...,0.533364,0.532377,0.526874,0.530893,0.519402,0.534972,0.543166,0.537226,0.528638,0.519520
3531,0.573871,0.530496,0.523590,0.532491,0.535035,0.526557,0.535377,0.517131,0.537258,0.532061,...,0.538687,0.537531,0.525161,0.532762,0.537737,0.547027,0.540337,0.534811,0.542981,0.521472
76532,0.573713,0.525386,0.528596,0.520002,0.533358,0.547112,0.527465,0.529487,0.535468,0.544843,...,0.529032,0.524485,0.519618,0.530465,0.558319,0.549647,0.554349,0.540310,0.559867,0.540053
60862,0.571175,0.562619,0.528209,0.526664,0.551450,0.544291,0.538256,0.539550,0.569515,0.555556,...,0.535454,0.541170,0.551795,0.530051,0.557067,0.549802,0.547064,0.528851,0.551774,0.534805


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
similar_custID=df_total_quantity.loc[df_total_quantity['CustomerID'].isin(top5.index.tolist())]
similar_custID.head()

,CustomerID,ProductName,Quantity
2891,3531,Baking Powder,1
2892,3531,"Bar Mix - Pina Colada, 355 Ml",1
2893,3531,Bay Leaf,1
2894,3531,Beans - Kidney White,1
2895,3531,"Beef - Chuck, Boneless",1


### Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
df_cust_grby= similar_custID.groupby(['ProductName'] , as_index=False).agg({'Quantity': 'sum'}).sort_values('Quantity', ascending= False)
df_cust_grby.head()

,ProductName,Quantity
108,Knife Plastic - White,64
154,Phyllo Dough,64
38,Cake - Box Window 10x10x2.5,52
185,"Soup - Campbells, Beef Barley",52
136,Onions - Cippolini,48


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
merged= df_cust_grby.merge(product_customer_matrix, left_on='ProductName', right_on='ProductName' )
merged_nobuy= merged[merged[33]==0]
merged_nobuy.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()

['Knife Plastic - White',
 'Soup - Campbells, Beef Barley',
 'Onions - Cippolini',
 'Tea - Herbal Sweet Dreams',
 'Muffin - Zero Transfat']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
reco_dict={}
customers=list(data['CustomerID'].unique())

for cust in customers:
    top5 = customer_similarity_matrix.nlargest(6, [cust])[1:6]
    similar_custID=df_total_quantity.loc[df_total_quantity['CustomerID'].isin(top5.index.tolist())]
    df_cust_grby= similar_custID.groupby(['ProductName'] , as_index=False).agg({'Quantity': 'sum'}).sort_values('Quantity', ascending= False)
    merged= df_cust_grby.merge(product_customer_matrix, left_on='ProductName', right_on='ProductName' )
    merged_nobuy= merged[merged[cust]==0]
    reco_list= merged_nobuy.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()
    reco_dict.update({cust: reco_list})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
df_recomended= pd.DataFrame(reco_dict)
df_recomended.head()

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Jagermeister,Bread - Rye,Juice - V8 Splash,Juice - Orange,Wine - Blue Nun Qualitatswein,Bouq All Italian - Primerba,"Nut - Chestnuts, Whole",Cheese Cloth No 100,Veal - Sweetbread,Foam Cup 6 Oz,...,"Wine - White, Colubia Cresh",Beef - Inside Round,Milk - 2%,Oil - Shortening - All - Purpose,Clam Nectar,Squid - Tubes / Tenticles 10/20,"Hickory Smoke, Liquid",Bread - Calabrese Baguette,Flavouring - Orange,Creme De Banane - Marie
1,Wine - Two Oceans Cabernet,Fondant - Icing,Beef - Inside Round,Bagel - Plain,Cookies - Assorted,Onions - Vidalia,"Beef - Tenderlion, Center Cut","Wine - Cahors Ac 2000, Clos",Ecolab - Lime - A - Way 4/4 L,Spinach - Baby,...,Scallops - 10/20,Grenadine,Bay Leaf,Foam Cup 6 Oz,Pastry - Raisin Muffin - Mini,"Pepper - White, Ground","Salsify, Organic",French Pastry - Mini Chocolate,Longos - Chicken Wings,"Wine - Red, Colio Cabernet"
2,Tofu - Firm,Lettuce - Frisee,"Bread - Roll, Soft White Round",Olives - Stuffed,Cheese - Cottage Cheese,Scallops - Live In Shell,Veal - Eye Of Round,Butter - Unsalted,Island Oasis - Mango Daiquiri,Cookies - Assorted,...,Wasabi Powder,Cheese - Victor Et Berthold,Banana Turning,"Spoon - Soup, Plastic","Spoon - Soup, Plastic",Soup Knorr Chili With Beans,Dc Hikiage Hira Huba,Cheese - Parmesan Grated,"Cheese - Brie, Triple Creme","Nut - Pistachio, Shelled"
3,Appetizer - Sausage Rolls,Pork - Kidney,Truffle Cups - Brown,Veal - Sweetbread,Baking Powder,Pastry - Choclate Baked,"Soup - Canadian Pea, Dry Mix","Bread - Roll, Canadian Dinner",Milk - 2%,Sardines,...,Cup - Translucent 7 Oz Clear,"Pepper - Paprika, Hungarian",Beer - Labatt Blue,Beef - Montreal Smoked Brisket,Pastry - Butterscotch Baked,Cheese - Bocconcini,Oil - Shortening - All - Purpose,Foam Dinner Plate,Tia Maria,Napkin White - Starched
4,Dc - Frozen Momji,Bread - French Baquette,Cake - Mini Cheesecake,Cinnamon Buns Sticky,Quiche Assorted,"Soup - Canadian Pea, Dry Mix",Wine - Gato Negro Cabernet,Onions - Cippolini,Vaccum Bag 10x13,Cheese - Parmesan Grated,...,Beef - Montreal Smoked Brisket,Wine - Fume Blanc Fetzer,Wine - Vidal Icewine Magnotta,Table Cloth - 53x69 Colour,Wine - Magnotta - Belpaese,Barramundi,Banana - Leaves,Cheese - Mix,Fondant - Icing,"Crab - Dungeness, Whole"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
reco_dict={}
customers=list(data['CustomerID'].unique())

for cust in customers:
    top5 = customer_similarity_matrix.nlargest(6, [cust])[1:6]
    similar_custID=df_total_quantity.loc[df_total_quantity['CustomerID'].isin(top5.index.tolist())]
    df_cust_grby= similar_custID.groupby(['ProductName'] , as_index=False).agg({'Quantity': 'sum'}).sort_values('Quantity', ascending= False)
    merged= df_cust_grby.merge(product_customer_matrix, left_on='ProductName', right_on='ProductName' )
    merged_nobuy= merged[merged[cust]==0]
    reco_list= merged_nobuy.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()
    reco_dict.update({cust: reco_list})

reco_dict

{61288: ['Jagermeister',
  'Wine - Two Oceans Cabernet',
  'Tofu - Firm',
  'Appetizer - Sausage Rolls',
  'Dc - Frozen Momji'],
 77352: ['Bread - Rye',
  'Fondant - Icing',
  'Lettuce - Frisee',
  'Pork - Kidney',
  'Bread - French Baquette'],
 40094: ['Juice - V8 Splash',
  'Beef - Inside Round',
  'Bread - Roll, Soft White Round',
  'Truffle Cups - Brown',
  'Cake - Mini Cheesecake'],
 23548: ['Juice - Orange',
  'Bagel - Plain',
  'Olives - Stuffed',
  'Veal - Sweetbread',
  'Cinnamon Buns Sticky'],
 78981: ['Wine - Blue Nun Qualitatswein',
  'Cookies - Assorted',
  'Cheese - Cottage Cheese',
  'Baking Powder',
  'Quiche Assorted'],
 83106: ['Bouq All Italian - Primerba',
  'Onions - Vidalia',
  'Scallops - Live In Shell',
  'Pastry - Choclate Baked',
  'Soup - Canadian Pea, Dry Mix'],
 11253: ['Nut - Chestnuts, Whole',
  'Beef - Tenderlion, Center Cut',
  'Veal - Eye Of Round',
  'Soup - Canadian Pea, Dry Mix',
  'Wine - Gato Negro Cabernet'],
 35107: ['Cheese Cloth No 100',
  'Wi

In [15]:
# Se aprecian cambios de algunos productos según que distancia usemos.
# No se debido a qué especificamente.
df_recomended= pd.DataFrame(reco_dict)
df_recomended.head()

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Jagermeister,Bread - Rye,Juice - V8 Splash,Juice - Orange,Wine - Blue Nun Qualitatswein,Bouq All Italian - Primerba,"Nut - Chestnuts, Whole",Cheese Cloth No 100,Veal - Sweetbread,Foam Cup 6 Oz,...,"Wine - White, Colubia Cresh",Beef - Inside Round,Milk - 2%,Oil - Shortening - All - Purpose,Clam Nectar,Squid - Tubes / Tenticles 10/20,"Hickory Smoke, Liquid",Bread - Calabrese Baguette,Flavouring - Orange,Creme De Banane - Marie
1,Wine - Two Oceans Cabernet,Fondant - Icing,Beef - Inside Round,Bagel - Plain,Cookies - Assorted,Onions - Vidalia,"Beef - Tenderlion, Center Cut","Wine - Cahors Ac 2000, Clos",Ecolab - Lime - A - Way 4/4 L,Spinach - Baby,...,Scallops - 10/20,Grenadine,Bay Leaf,Foam Cup 6 Oz,Pastry - Raisin Muffin - Mini,"Pepper - White, Ground","Salsify, Organic",French Pastry - Mini Chocolate,Longos - Chicken Wings,"Wine - Red, Colio Cabernet"
2,Tofu - Firm,Lettuce - Frisee,"Bread - Roll, Soft White Round",Olives - Stuffed,Cheese - Cottage Cheese,Scallops - Live In Shell,Veal - Eye Of Round,Butter - Unsalted,Island Oasis - Mango Daiquiri,Cookies - Assorted,...,Wasabi Powder,Cheese - Victor Et Berthold,Banana Turning,"Spoon - Soup, Plastic","Spoon - Soup, Plastic",Soup Knorr Chili With Beans,Dc Hikiage Hira Huba,Cheese - Parmesan Grated,"Cheese - Brie, Triple Creme","Nut - Pistachio, Shelled"
3,Appetizer - Sausage Rolls,Pork - Kidney,Truffle Cups - Brown,Veal - Sweetbread,Baking Powder,Pastry - Choclate Baked,"Soup - Canadian Pea, Dry Mix","Bread - Roll, Canadian Dinner",Milk - 2%,Sardines,...,Cup - Translucent 7 Oz Clear,"Pepper - Paprika, Hungarian",Beer - Labatt Blue,Beef - Montreal Smoked Brisket,Pastry - Butterscotch Baked,Cheese - Bocconcini,Oil - Shortening - All - Purpose,Foam Dinner Plate,Tia Maria,Napkin White - Starched
4,Dc - Frozen Momji,Bread - French Baquette,Cake - Mini Cheesecake,Cinnamon Buns Sticky,Quiche Assorted,"Soup - Canadian Pea, Dry Mix",Wine - Gato Negro Cabernet,Onions - Cippolini,Vaccum Bag 10x13,Cheese - Parmesan Grated,...,Beef - Montreal Smoked Brisket,Wine - Fume Blanc Fetzer,Wine - Vidal Icewine Magnotta,Table Cloth - 53x69 Colour,Wine - Magnotta - Belpaese,Barramundi,Banana - Leaves,Cheese - Mix,Fondant - Icing,"Crab - Dungeness, Whole"
